In [1]:
import numpy as np
###############################################################################################################
###########################################  Main MD Program     ##############################################
###############################################################################################################

def run_md(N, L, n_steps, m, T, dump, dt):
    """
    MD program using velocity verlet algorithm.
    
    N       - integer number of atoms
    L       - float length of side of cubic box
    n_steps - integer number of MD steps
    m       - float scalar of mass of all particles
    T       - float temperature for initializing velocities
    dump    - integer write frequency in steps
    dt      - float integration time step
    """
    
    # initialize positions
    x = initialize_positions(N, L)
    # initialize velocities
    v = initialize_velocities(N, m, T)
    # compute initial forces
    #f0 = compute_forces(x, L)
    
    # open trajectory file
    traj_file = open("traj_gas.xyz", 'w')
    
    for step in range(n_steps):
    
        # propagate positions
        velocity_verlet_update_positions(x,v,m,dt,L)
    
        # compute forces at time t+dt
        #f1 = compute_forces(x, L)
        
        # propagate velocities
        velocity_verlet_update_velocities(v,m,dt)
        
        # save old forces
        #f0 = np.copy(f1)
        
        if (step%dump==0):
            write_trajectory_frame(x, traj_file, step)
    
    # close trajectory file
    traj_file.close()
            
    
###############################################################################################################
###########################################  Subroutines     ##################################################
###############################################################################################################

# initialize positions
def initialize_positions(N, L):
    """
    N - integer number of particles
    L - float size of box
    """

    return np.random.rand(N,3)*L

# initialize velocities
def initialize_velocities(N, m, T):
    """
    N - integer number of particles
    m - float mass of particles
    T - float target temperature for Maxwell-Boltzmann distribution
    """
    return np.random.normal(size=(N,3))*np.sqrt(T/m)
    
# velocity verlet algorithm and wrap particles in PBC
def velocity_verlet_update_positions(x, v, m, dt, L):
    """
    x     - (n_atoms, 3) float array of particle positions
    v     - (n_atoms, 3) float array of particle velocities
    f     - (n_atoms, 3) float array of particle forces
    m     - float of particle mass - all particles have the same mass
    dt    - float of integration time step
    L     - float of box length
    """
    
    N = x.shape[0]
    
    x += v*dt + 0.5*dt*dt
    
    # wrap into central box (box is from 0 to L in each dimension)
    for i in range(N):
        for j in range(3):
            if x[i,j] < 0:
                x[i,j] += L
            elif x[i,j] > L:
                x[i,j] -= L
    

# velocity verlet algorithm and wrap particles in PBC
def velocity_verlet_update_velocities(v, m, dt):
    """
    v     - (n_atoms, 3) float array of particle velocities
    f0    - (n_atoms, 3) float array of particle forces at time t
    f1    - (n_atoms, 3) float array of particle forces at time t + dt
    m     - float of particle mass - all particles have the same mass
    dt    - float of integration time step
    """
    #v += v+ + dt*(f0 + f1)/(2*m)
    
    v += 0.5*dt
        
def write_trajectory_frame(x, file_pointer, step):
    """
    x             - (n_atoms, 3) float array of particle positiosn
    file_pointer  - trajectory file pointer
    step          - integer step number
    """
    
    N = x.shape[0]
    
    file_pointer.write("%10d\n" % (N))
    file_pointer.write("%10d\n" % (step))
    for i in range(N):
        file_pointer.write("C   %10.5f %10.5f %10.5f\n" % (x[i,0], x[i,1], x[i,2]))

In [2]:
#test sim
N=2
L=10
dump=20
n_steps=200*dump
m=12 #carbon in g/mol
T=0.5 #Requested T
dt=0.002
out=run_md(N, L, n_steps, m, T, dump, dt)